In [1]:
require('nn')
require('hdf5')
require('optim')

In [2]:
f = hdf5.open("data.hdf5", "r")
X_train = f:read("train_input"):all()
Y_train = f:read("train_output"):all()
X_valid = f:read("valid_input"):all()
q_valid = f:read("valid_q"):all()
X_test = f:read("test_input"):all()
q_test = f:read("test_q"):all()
kaggle_valid = f:read("valid_kaggle"):all()
nclasses = f:read('nclasses'):all():long()[1]
nfeatures = f:read('nfeatures'):all():long()[1]
f:close()

window_size = X_train:size(2)
n_actual = nfeatures

In [8]:
function countMatrix(X, Y, C, K)
    CM = torch.zeros(C, K)
    for i=1,X:size(1) do
        for j=1, X:size(2) do
            local row = Y[i]
            local col = X[i][j]
            CM[row][col] = CM[row][col] + 1
        end
    end
    return CM
end

function smooth(matrix, alpha)
    return matrix:add(alpha)
end

function normalize(matrix)
    return matrix:cdiv(torch.expand(matrix:sum(2), matrix:size(1), matrix:size(2)))
end

function predictDistribution(matrix, X)
    jll = torch.ones(X:size(1), matrix:size(2))
    for i=1,X:size(1) do
        temp = torch.ones(matrix:size(2))
        for j=1,X:size(2) do
            temp:cmul(matrix:select(2, X[i][j]))
        end
        jll:sub(i,i):cmul(torch.reshape(temp, 1, temp:size(1)))
    end
    return jll
end

In [3]:
function valid_perplexity(jll, Y_correct, Y)
    jll:cdiv(torch.expand((torch.sum(jll, 2)), Y:size(1), jll:size(2)))
    perp = 0
    for i=1, Y_correct:size(1) do
        idx = Y[i][Y_correct[i]]
        perp = (torch.log(jll[i][idx])) + perp
    end
    
    print(torch.exp(perp/Y:size(1) * -1))
end

function kaggle_perplexity(jll, Y)
    local matrix = torch.zeros(Y:size(1), Y:size(2))
    for i=1, Y:size(1) do
        for j=1, Y:size(2) do
            idx = Y[i][j]
            matrix[i][j] = (jll[i][idx] * -1)
        end
    end
    return matrix:cdiv(torch.expand((torch.sum(matrix, 2)), Y:size(1), matrix:size(2)))
end

function write2file(scores, fname)
    f = io.open(fname, "w")
    f:write("ID,Class1,CLass2,Class3,Class4,Class5,Class6,Class7,Class8,Class9,Class10,Class11,Class12,CLass13,Class14,Class15,Class16,Class17,Class18,Class19,Class20,Class21,Class22,Class23,CLass24,Class25,Class26,Class27,Class28,Class29,Class30,Class31,Class32,Class33,Class34,CLass35,Class36,Class37,Class38,Class39,Class40,Class41,Class42,Class43,Class44,Class45,CLass46,Class47,Class48,Class49,Class50\n")
    for i=1,scores:size(1) do
        s = tostring(i)
        for j=1, scores:size(2) do
            s = s .. "," .. tostring(scores[i][j])
        end
        f:write(s .. "\n")
    end
    f:close()
end

In [4]:
CM = countMatrix(X_train, Y_train, nfeatures, nfeatures)

In [5]:
CM = smooth(CM, 1)

In [6]:
CM = normalize(CM)

In [10]:
jll = predictDistribution(CM, X_valid)

In [12]:
valid_perplexity(jll, kaggle_valid, q_valid)

2304.994812898	



In [77]:
scores = kaggle_perplexity(jll, q_test)
write2file(scores, "pred_4.csv")

In [15]:
norm_jll = jll:cdiv(torch.expand((torch.sum(jll, 2)), jll:size(1), jll:size(2)))

1	
